# per Tissue Analysis and Models
*Ben Imlay*

## Database Setup

In [31]:
import pandas as pd
from pathlib import Path
from os import listdir
import sklearn.model_selection
import sklearn.feature_selection
TISSUE='Liver'

In [32]:
data_dir=Path("data")
tissue_dir=Path("tissue-specific")
manifest={"data":"All_Tissue_Site_Details.combined.reads.gct",
              "sample_meta":"GTEx_v7_Annotations_SampleAttributesDS.txt",
              "subject_meta":"GTEx_v7_Annotations_SubjectPhenotypesDS.txt",
               "merged_meta":"merged_meta.tsv"}
meta=pd.read_csv(data_dir/manifest['merged_meta'],sep="\t",dtype={'SMUBRID':object,'SEX':object,'DTHHRDY':object})
meta=meta[~(meta['AGE'].isnull())] # removes all samples without age
meta.iloc[0:3,:]

SAMPID  SMATSSCR SMCENTER  \
0  GTEX-1117F-0226-SM-5GZZ7       0.0       B1   
1  GTEX-111CU-1826-SM-5GZYN       0.0       B1   
2  GTEX-111FC-0226-SM-5N9B8       2.0       B1   

                                            SMPTHNTS  SMRIN            SMTS  \
0       2 pieces, ~15% vessel stroma, rep delineated    6.8  Adipose Tissue   
1  2 pieces, small portion of nerve (<10% of one ...    7.5  Adipose Tissue   
2  2 pieces, larger piece is 30% fibrovascular ti...    7.3  Adipose Tissue   

                    SMTSD  SMUBRID  SMTSISCH  SMTSPAX   ...      SME1PCTS  \
0  Adipose - Subcutaneous  0002190    1214.0   1125.0   ...     50.094357   
1  Adipose - Subcutaneous  0002190     138.0    571.0   ...     50.089250   
2  Adipose - Subcutaneous  0002190    1040.0    869.0   ...     50.113018   

   SMRRNART  SME1MPRT SMNUM5CD SMDPMPRT  SME2PCTS      SUBJID  SEX    AGE  \
0  0.003102  0.992826      NaN      0.0  50.12628  GTEX-1117F  2.0  60-69   
1  0.002126  0.993923      NaN      0.0  50.10142  GTEX-111CU  1.0  50-59   
2  0.003756  0.992825      NaN      0.0  50.06186  GTEX-111FC  1.0  60-69   

   DTHHRDY  
0      4.0  
1      0.0  
2      1.0  

[3 rows x 67 columns]

## Avaiable Tissues

In [33]:
# To do python plots of counts
infiles=listdir(data_dir/tissue_dir)
TISSUE_files=[f for f in infiles if  TISSUE in f]
TISSUE_files

['Liver_lcpm.tsv', 'Liver_cpm.tsv']

## Reading CPM and LCPM files.

In [37]:
cpm=pd.read_csv(data_dir/tissue_dir/TISSUE_files[1],sep="\t",index_col=0)
lcpm=pd.read_csv(data_dir/tissue_dir/TISSUE_files[0],sep="\t",index_col=0)

In [36]:
cpm.iloc[0:5,0:20]

ENSG00000223972  ENSG00000227232  ENSG00000243485  \
GTEX-11DXY-0526-SM-5EGGQ         0.142941         9.691384         0.000000   
GTEX-11DXZ-0126-SM-5EGGY         0.175082         7.633580         0.140066   
GTEX-11EQ9-0526-SM-5A5JZ         0.000000         3.867529         0.021606   
GTEX-11GSP-0626-SM-5986T         0.133690        14.505370         0.033423   
GTEX-11NUK-1226-SM-5P9GM         0.174075         9.008397         0.152316   

                          ENSG00000237613  ENSG00000268020  ENSG00000240361  \
GTEX-11DXY-0526-SM-5EGGQ         0.085764         0.000000         0.114353   
GTEX-11DXZ-0126-SM-5EGGY         0.000000         0.070033         0.140066   
GTEX-11EQ9-0526-SM-5A5JZ         0.064819         0.021606         0.000000   
GTEX-11GSP-0626-SM-5986T         0.100268         0.000000         0.133690   
GTEX-11NUK-1226-SM-5P9GM         0.021759         0.043519         0.043519   

                          ENSG00000186092  ENSG00000238009  ENSG00000233750  \
GTEX-11DXY-0526-SM-5EGGQ         0.000000         0.171529         0.714704   
GTEX-11DXZ-0126-SM-5EGGY         0.070033         0.490230         0.665312   
GTEX-11EQ9-0526-SM-5A5JZ         0.021606         0.043213         0.237669   
GTEX-11GSP-0626-SM-5986T         0.100268         0.133690         0.401070   
GTEX-11NUK-1226-SM-5P9GM         0.108797         0.130556         0.652782   

                          ENSG00000237683  ENSG00000268903  ENSG00000239906  \
GTEX-11DXY-0526-SM-5EGGQ         8.204800         0.000000         0.000000   
GTEX-11DXZ-0126-SM-5EGGY         1.785838         0.000000         0.000000   
GTEX-11EQ9-0526-SM-5A5JZ         3.370584         0.043213         0.021606   
GTEX-11GSP-0626-SM-5986T         5.046799         0.033423         0.167113   
GTEX-11NUK-1226-SM-5P9GM         4.373642         0.021759         0.065278   

                          ENSG00000241860  ENSG00000222623  ENSG00000241599  \
GTEX-11DXY-0526-SM-5EGGQ         0.857645              0.0         0.028588   
GTEX-11DXZ-0126-SM-5EGGY         0.420197              0.0         0.035016   
GTEX-11EQ9-0526-SM-5A5JZ         0.453732              0.0         0.000000   
GTEX-11GSP-0626-SM-5986T         0.668450              0.0         0.100268   
GTEX-11NUK-1226-SM-5P9GM         0.652782              0.0         0.021759   

                          ENSG00000228463  ENSG00000237094  ENSG00000250575  \
GTEX-11DXY-0526-SM-5EGGQ         2.172700         0.343058         0.000000   
GTEX-11DXZ-0126-SM-5EGGY         0.770361         0.420197         0.035016   
GTEX-11EQ9-0526-SM-5A5JZ         0.345701         0.216063         0.021606   
GTEX-11GSP-0626-SM-5986T         1.102943         0.969253         0.033423   
GTEX-11NUK-1226-SM-5P9GM         1.784272         0.674542         0.043519   

                          ENSG00000233653  ENSG00000224813  
GTEX-11DXY-0526-SM-5EGGQ         0.200117                0  
GTEX-11DXZ-0126-SM-5EGGY         0.385181                0  
GTEX-11EQ9-0526-SM-5A5JZ         0.064819                0  
GTEX-11GSP-0626-SM-5986T         0.267380                0  
GTEX-11NUK-1226-SM-5P9GM         0.369910                0

In [35]:
lcpm.iloc[0:5,0:20]

ENSG00000223972  ENSG00000227232  ENSG00000243485  \
GTEX-11DXY-0526-SM-5EGGQ        -2.513117         3.281493        -4.955184   
GTEX-11DXZ-0126-SM-5EGGY        -2.270082         2.938439        -2.536991   
GTEX-11EQ9-0526-SM-5A5JZ        -4.955184         1.963387        -4.215116   
GTEX-11GSP-0626-SM-5986T        -2.591388         3.861718        -3.928874   
GTEX-11NUK-1226-SM-5P9GM        -2.277105         3.176424        -2.437902   

                          ENSG00000237613  ENSG00000268020  ENSG00000240361  \
GTEX-11DXY-0526-SM-5EGGQ        -3.083136        -4.955184        -2.770155   
GTEX-11DXZ-0126-SM-5EGGY        -4.955184        -3.289562        -2.536991   
GTEX-11EQ9-0526-SM-5A5JZ        -3.365055        -4.215116        -4.955184   
GTEX-11GSP-0626-SM-5986T        -2.915897        -4.955184        -2.591388   
GTEX-11NUK-1226-SM-5P9GM        -4.211019        -3.722519        -3.722519   

                          ENSG00000186092  ENSG00000238009  ENSG00000233750  \
GTEX-11DXY-0526-SM-5EGGQ        -4.955184        -2.295023        -0.420936   
GTEX-11DXZ-0126-SM-5EGGY        -3.289562        -0.936591        -0.519636   
GTEX-11EQ9-0526-SM-5A5JZ        -4.215116        -3.728362        -1.889474   
GTEX-11GSP-0626-SM-5986T        -2.915897        -2.591388        -1.206541   
GTEX-11NUK-1226-SM-5P9GM        -2.825895        -2.618894        -0.545786   

                          ENSG00000237683  ENSG00000268903  ENSG00000239906  \
GTEX-11DXY-0526-SM-5EGGQ         3.042125        -4.955184        -4.955184   
GTEX-11DXZ-0126-SM-5EGGY         0.862410        -4.955184        -4.955184   
GTEX-11EQ9-0526-SM-5A5JZ         1.766731        -3.728362        -4.215116   
GTEX-11GSP-0626-SM-5986T         2.344554        -3.928874        -2.326635   
GTEX-11NUK-1226-SM-5P9GM         2.139429        -4.211019        -3.358243   

                          ENSG00000241860  ENSG00000222623  ENSG00000241599  \
GTEX-11DXY-0526-SM-5EGGQ        -0.168316        -4.955184        -4.039212   
GTEX-11DXZ-0126-SM-5EGGY        -1.144224        -4.955184        -3.894270   
GTEX-11EQ9-0526-SM-5A5JZ        -1.041065        -4.955184        -4.955184   
GTEX-11GSP-0626-SM-5986T        -0.513160        -4.955184        -2.915897   
GTEX-11NUK-1226-SM-5P9GM        -0.545786        -4.955184        -4.211019   

                          ENSG00000228463  ENSG00000237094  ENSG00000250575  \
GTEX-11DXY-0526-SM-5EGGQ         1.140736        -1.413908        -4.955184   
GTEX-11DXZ-0126-SM-5EGGY        -0.317252        -1.144224        -3.894270   
GTEX-11EQ9-0526-SM-5A5JZ        -1.403783        -2.009849        -4.215116   
GTEX-11GSP-0626-SM-5986T         0.182920         0.002146        -3.928874   
GTEX-11NUK-1226-SM-5P9GM         0.861167        -0.500672        -3.722519   

                          ENSG00000233653  ENSG00000224813  
GTEX-11DXY-0526-SM-5EGGQ        -2.105610        -4.955184  
GTEX-11DXZ-0126-SM-5EGGY        -1.260440        -4.955184  
GTEX-11EQ9-0526-SM-5A5JZ        -3.365055        -4.955184  
GTEX-11GSP-0626-SM-5986T        -1.738813        -4.955184  
GTEX-11NUK-1226-SM-5P9GM        -1.314209        -4.955184

### Merging metadata

In [28]:
tissue_meta=meta[meta['SMTSD']==TISSUE]
tissue_meta.iloc[0:5]

SAMPID  SMATSSCR SMCENTER  \
6536  GTEX-11DXY-0526-SM-5EGGQ       1.0       B1   
6537  GTEX-11DXZ-0126-SM-5EGGY       1.0       B1   
6538  GTEX-11EQ9-0526-SM-5A5JZ       1.0       B1   
6539  GTEX-11GSP-0626-SM-5986T       1.0       B1   
6540  GTEX-11NUK-1226-SM-5P9GM       1.0       B1   

                                               SMPTHNTS  SMRIN   SMTS  SMTSD  \
6536                               2 pieces, congestion    6.8  Liver  Liver   
6537  2 pieces, foci of hepatocyte necrosis and ball...    7.9  Liver  Liver   
6538  2 pieces, diffuse macro and microvesucular ste...    8.1  Liver  Liver   
6539              2 pieces; central vascular congestion    6.2  Liver  Liver   
6540  2 pieces, includes capsule (target is 1 cm bel...    6.1  Liver  Liver   

      SMUBRID  SMTSISCH  SMTSPAX   ...      SME1PCTS  SMRRNART  SME1MPRT  \
6536  0001114     891.0   1149.0   ...     50.825165  0.015545  0.993035   
6537  0001114     250.0    951.0   ...     50.576138  0.005714  0.993494   
6538  0001114      82.0    617.0   ...     50.221200  0.007967  0.995574   
6539  0001114     771.0    565.0   ...     50.910202  0.025063  0.992580   
6540  0001114     956.0    829.0   ...     49.882460  0.015518  0.989904   

     SMNUM5CD SMDPMPRT   SME2PCTS      SUBJID  SEX    AGE  DTHHRDY  
6536      NaN      0.0  49.517372  GTEX-11DXY  1.0  60-69      2.0  
6537      NaN      0.0  49.746162  GTEX-11DXZ  1.0  50-59      0.0  
6538      NaN      0.0  49.962532  GTEX-11EQ9  1.0  30-39      2.0  
6539      NaN      0.0  49.414272  GTEX-11GSP  2.0  60-69      2.0  
6540      NaN      0.0  50.553600  GTEX-11NUK  1.0  50-59      2.0  

[5 rows x 67 columns]

## Test and Train Set Creation

In [48]:
cpm_train, cpm_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(cpm, tissue_meta['AGE'], test_size=.3, random_state=1234) # random state guarantees that the same split is made for a given tissue.
print(cpm_train.shape)
print(cpm_test.shape)
print(y_train.shape)
print(y_test.shape)
sum(cpm_train.iloc[:,0]) # Confirms that the split is the same each time

(122, 56202)
(53, 56202)
(122,)
(53,)


16.175560121040135

## Filter by Expression
As adapted from the edgeR package function [filterByExpression](https://rdrr.io/bioc/edgeR/src/R/filterByExpr.R).

In [40]:
def filter_by_expr(min):
    pass

(122, 56202)
(53, 56202)
(122,)
(53,)
